In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline
import os
import sys
import math
import glob
import shutil
import tempfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18}) # defaults to 10.0

In [ ]:
def ffmpeg(basename, outname):
    cmd = "ffmpeg -y -i %s_%%04d.png -vf \"scale=trunc(iw/2)*2:trunc(ih/2)*2\" -pix_fmt yuv420p %s" % (basename, outname)
    os.system(cmd)

In [ ]:
def animate(directory, name):
    tmp = tempfile.mkdtemp()
    files = glob.glob(os.path.join(directory, name) + "_????.csv")
    files.sort()
    print(tmp)

    for file in files:
        base = os.path.basename(file)
        
        df = pd.read_csv(file)
        fig = plt.figure(figsize=(12,9))
        ax = fig.gca()
        x = df['x']
        ax.plot(x,df['theta'], 'r-',linewidth=3.0, label=r'$\theta$')
        ax.plot(x,df['Upsilon'], 'b-',linewidth=3.0, label=r'$\Upsilon$')
        plt.title(name)
        ax.legend(loc='center right')
        ax.set_xlabel('Position')
        plt.savefig(os.path.join(tmp, base[:-3] + "png"), dpi=150, bbox_inches='tight')
        plt.close('all')

    # make video
    ffmpeg(os.path.join(tmp, name), name + ".mp4")

    # cleanup
    shutil.rmtree(tmp)

In [ ]:
animate("out", "run_1_1_out")

In [ ]:
animate("out", "run_2_1_out")

In [ ]:
animate("out", "run_4_1_out")

In [ ]:
animate("out", "run_8_1_out")

In [ ]:
animate("out", "run_2_0.1_out")

In [ ]:
animate("out", "run_4_2_out")

In [ ]:
animate("out", "run_2.0_2.0_432_out")

In [ ]:
animate("out", "run_2.0_1.2_432_out")

In [ ]:
def find_limits(a1, a2, t):
    # error check
    if len(a1) != len(a2):
        return None
    
    l0 = None
    l1 = None
    # lower limit
    if a2[0] > t:
        l0 = a1[0]
    else:
        for i in range(len(a1)-1):
            if a2[i+1] > t:
                f = (t-a2[i])/(a2[i+1]-a2[i])
                l0 = f*(a1[i+1]-a1[i]) + a1[i]
                break
    #upper limit
    if a2[-1] > t:
        l1 = a1[-1]
    else:
        for i in range(len(a1)-1):
            if a2[-2-i] > t:
                f = (t-a2[-1-i])/(a2[-2-i]-a2[-1-i])
                l1 = f*(a1[-2-i]-a1[-1-i]) + a1[-1-i]
                break

    # return limits
    if l0 is None or l1 is None:
        return None
    return (l0, l1)

In [ ]:
def plot_files(files):
    files.sort()

    fig = plt.figure(figsize=(12,9))
    ax = fig.gca()
    ax.set_xlabel('Position')
    ax.xlim(12,18)

    for file in files:
        df = pd.read_csv(file)
        x = df['x']
        ax.plot(x,df['Upsilon'], '-', linewidth=3.0, label=file)
        ax.plot(x,df['theta'], '.', linewidth=3.0, label=file)
        ax.legend(loc='center right')
        
#     plt.savefig(os.path.join(tmp, base[:-3] + "png"), dpi=150, bbox_inches='tight')
#     plt.close('all')

In [ ]:
def plot_glob(g):
    files = glob.glob(g)
    plot_files(files)

In [ ]:
plot_files(["out/run_2.0_1.2_432_out_0042.csv", "out/run_2.0_2.0_432_out_0038.csv"])

In [ ]:
# animation
tmp = tempfile.mkdtemp()
print (tmp)

# animation 2
for frame in range(235):
    marmot = np.transpose(np.loadtxt("multiphase_diffusion2_out_var_%04d.csv" % frame, delimiter=",", skiprows=1))
    fig = plt.figure(figsize=(12,9))
    ax = fig.gca()
    # phases
    ax.plot(marmot[0],marmot[9], 'r-',linewidth=3.0, label=r'$\eta_1$ $\delta$-UZr')
    ax.plot(marmot[0],marmot[10], 'g-',linewidth=3.0, label=r'$\eta_2$ $\alpha$-U')
    ax.plot(marmot[0],marmot[11], 'b-',linewidth=3.0, label=r'$\eta_3$ $\alpha$-Zr')
    # ranges
    #     range_delta = find_limits(marmot[0], marmot[9], 0.5)
    #     range_U     = find_limits(marmot[0], marmot[10], 0.5)
    #     range_Zr    = find_limits(marmot[0], marmot[11], 0.5)
    #     if range_delta is not None:
    #         ax.axvspan(range_delta[0], range_delta[1], alpha=0.25, color='red')
    #     if range_U is not None:
    #         ax.axvspan(range_U[0], range_U[1], alpha=0.25, color='green')
    #     if range_Zr is not None:
    #         ax.axvspan(range_Zr[0], range_Zr[1], alpha=0.25, color='blue')
    # concentrations
    ax.plot(marmot[0],marmot[8], 'y-',linewidth=3.0, label='c')
    ax.plot(marmot[0],marmot[4], 'r--',linewidth=3.0, label=r'$c_{1a}$')
    ax.plot(marmot[0],marmot[5], 'b--',linewidth=3.0, label=r'$c_{1b}$')
    plt.title(r'UZr $\delta$-phase growth')
    plt.xlim(-24.5,24.5)
    ax.legend(loc='upper left', bbox_to_anchor=(0.05,0.9))
    ax.set_xlabel('Position')
    ax.set_ylabel('Order parameter')
    plt.savefig(os.path.join(tmp, "multiphase_diffusion2_out2_var_%04d.png" % frame), dpi=150, bbox_inches='tight')
    plt.close('all')

# make video
#ffmpeg(os.path.join(tmp, "multiphase_diffusion2_out2_var"), "multiphase_diffusion2_phases.mp4")
             
# cleanup
#shutil.rmtree(tmp)

In [ ]:
tmp = tempfile.mkdtemp()
#patterns = ["out/steady_theta_2.0_2.0_432_sol_%04d.csv", "out/steady_theta_2.0_2.0_432_ana_%04d.csv"]
patterns = ["out/steady_upsilon_2.0_2.0_532.14_sol_%04d.csv", "out/steady_upsilon_2.0_2.0_532.14_ana_%04d.csv"]


xcols = ['x', 'x']
# ycols = ['theta', 'theta']
ycols = ['Upsilon', 'Upsilon']
labels = ['numerical', 'analytical']
line = ['-r', '--b']

png = "out_%04d.png"
mp4 = "steady2.mp4"
n = 1

while True:
    d = []
    for p in patterns:
        file = p % n
        if not os.path.isfile(file):
            break
        # load data
        d.append(pd.read_csv(file))
    else:
        # do all work with loaded data
        print("render")
        n += 1

        fig = plt.figure(figsize=(12,9))
        ax = fig.gca()
        for i in range(len(d)):
            ax.plot(d[i][xcols[i]], d[i][ycols[i]], line[i],linewidth=3.0, label=labels[i])
    
        #         plt.title(name)
        ax.legend(loc='center right')
        ax.set_xlabel('Position')
        plt.savefig(os.path.join(tmp, png % n), dpi=150, bbox_inches='tight')
        plt.close('all')

        continue
    break
        
# make video
cmd = "ffmpeg -y -i %s -vf \"scale=trunc(iw/2)*2:trunc(ih/2)*2\" -pix_fmt yuv420p %s" % (os.path.join(tmp, png), mp4)
os.system(cmd)

# cleanup
shutil.rmtree(tmp)

In [ ]:
files1 = glob.glob("out/velocity_theta_1.933_1_???.csv")
files1.sort()

x1 = []
y1 = []
ay1 = []
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    x1.append(T)
    y1.append(np.average(list(d['dpos'][-20:]/d['dt'][-20:]))*1e-9)
    ay1.append(np.average(list(d['vanalytic'][-20:]))*1e-9)

m1,b1 = np.polyfit(x1, ay1, 1) 

files2 = glob.glob("out/velocity_upsilon_1.933_1_???.csv")
files2.sort()

x2 = []
y2 = []
ay2 = []
for f in files2:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    x2.append(T)
    y2.append(np.average(list(d['dpos'][-100:]/d['dt'][-100:]))*1e-9)
    ay2.append(np.average(list(d['vanalytic'][-100:]))*1e-9)

m2,b2 = np.polyfit(x2, ay2, 1) 

fig = plt.figure(figsize=(12,9))
ax = fig.gca()

ax.plot(x1, y1, 'o',linewidth=3.0, label=r'$v_\vartheta$ numerical')
ax.plot(x1, m1*np.array(x1)+b1, '-', label=r'$v_\vartheta$ analytical')

ax.plot(x2, y2, 'o',linewidth=3.0, label=r'$v_\Upsilon$ numerical')
ax.plot(x2, m2*np.array(x2)+b2, '-', label=r'$v_\Upsilon$ analytical')

ax.legend(loc='lower right')
ax.set_xlabel('Temperature [K]')
ax.set_ylabel('Velocity [m/s]')

plt.savefig('velocity.png', dpi=150, bbox_inches='tight')
#plt.close('all')

In [ ]:
files1 = glob.glob("out/velocity_theta_1.933_1_???.csv")
files1.sort()

fig = plt.figure(figsize=(12,9))
ax = fig.gca()
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    ax.plot(d['dpos']/d['dt'], '*',label="T=%d" % T)

ax.legend(loc='upper right')
ax.set_xlabel('step')
ax.set_ylabel('Velocity [m/s]')


In [ ]:
files1 = glob.glob("out/velocity_upsilon_1.933_1_???.csv")
files1.sort()

fig = plt.figure(figsize=(12,9))
ax = fig.gca()
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    ax.plot(d['dpos']/d['dt'], '*',label="T=%d" % T)

ax.legend(loc='lower right')
ax.set_xlabel('step')
ax.set_ylabel('Velocity [m/s]')


In [ ]:
files1 = glob.glob("out/velocity_theta_1.933_1_392_op_*.csv")
files1.sort()

fig = plt.figure(figsize=(22,9))
ax = fig.gca()
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    ax.plot(d['x'], d['theta'], 'r')
    ax.plot(d['x'], d['Upsilon'], 'b')


In [ ]:
files1 = glob.glob("out/velocity_upsilon_1.933_1_392_op_*0.csv")
files1.sort()

fig = plt.figure(figsize=(22,9))
ax = fig.gca()
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    ax.plot(d['x'], d['theta'], 'r')
    ax.plot(d['x'], d['Upsilon'], 'b')
